In [2]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
import zipfile
import scipy.io as sio
import torch.utils.data as data
import glob

class UCSD(data.Dataset):
    """UCSD pedestrian counting data."""

    def __init__(self, data_dir, annotation_dir, transform=None):

        self.file_list = []
        self.file_cnts = []
        
        files = glob.glob(
            os.path.join(
                annotation_dir, 
                '*count_roi_mainwalkway.mat'))
        
        for f in files:
            tmp = sio.loadmat(f)
            
            l_count = tmp['count'][0][0].ravel()
            r_count = tmp['count'][0][1].ravel()
            t_count = l_count + r_count
            
            [self.file_cnts.append(c) for c in t_count]
            
            file_parts = os.path.basename(f).split('_')
            seq_id = "_".join(file_parts[0:3])
            
            for i in np.arange(len(t_count)):
                
                img_name = os.path.join(
                        data_dir,
                        seq_id + ".y",
                        "{}_f{:03d}.png".format(seq_id,i+1))
                
                img = Image.open(img_name)
                img = img.resize((128,128))
                
                self.file_list.append(img)
            
            self.transform = transform

    def __len__(self):
        return len(self.file_list)
        
    def __getitem__(self, idx):
        img = self.file_list[idx]

        if self.transform:
            img = self.transform(img)
        return img, self.file_cnts[idx]

In [3]:
import torch
import random

class SubsetSampler(object):

    def __init__(self, subset):
        self.subset = subset

    def __iter__(self):
        return iter(self.subset)

    def __len__(self):
        return len(self.subset)


class RandomSubsetSampler(object):

    def __init__(self, data_source, train_share=0.8):

        # Generate a list of indizes reaching from 0 ... len(data_source)-1
        idxList = list(range(0,len(data_source)))

        # Ensure that list is sorted randomly
        random.shuffle(idxList)

        # Split dataset random shares of train and test data
        numberOfTrainSamples = int(len(data_source) / (1 / train_share))
        
        self.train_samples = idxList[:numberOfTrainSamples]
        self.test_samples = idxList[numberOfTrainSamples:]


    def trainSampler(self):
        return SubsetSampler(self.train_samples)

    def testSampler(self):
        return SubsetSampler(self.test_samples)

In [4]:
import numpy as np
import torch

# Converts the 1-channel image into a 3-channel mage
class ExpandTo3D(object):

    def __call__(self, image):

        width = image.size()[1]
        height = image.size()[2]

        return image.expand(3,width,height)

In [5]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo


class CountingCNN(nn.Module):

    def __init__(self, regression_layer):
        super(CountingCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.regression_layer = regression_layer

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.regression_layer(x)
        return x


In [6]:
# We define different Regression Output Layer and evaluate their performance
regression_layer1 = nn.Sequential(
    nn.Dropout(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.Linear(4096, 4096),
    nn.Linear(4096, 1),
)

In [7]:
def evaluate_cnn_performance(net, max_epoch=150):
    
    learningRate = 0.00001

    performance_statistic = []

    criterion = nn.MSELoss()
    optimizer = optim.SGD(net.parameters(), lr=learningRate)

    print("Start training")
    
    for epoch in range(max_epoch):

        running_loss = 0.0
        for data in trainloader:

            # get the inputs
            inputs, labels = data
            labels = labels.float()

            inputs = inputs.cuda()
            labels = labels.cuda()

            labels = labels.view(-1,1)

            # wrap them in Variable
            inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)

            loss = criterion(outputs, labels)
            loss.backward()

            optimizer.step()

            # Keep track of the loss for reporting purpose
            running_loss += loss.data[0]

        if epoch % 50 == 0:
            learningRate /= 10
            optimizer = optim.SGD(net.parameters(), lr=learningRate)

        print( str(epoch) + ": " + str(running_loss))

        mae = 0.0
        total_diff = 0.0
        total_nr = 0

        for data in testloader:

            inputs, labels = data
            labels = labels.float()

            inputs = inputs.cuda()
            labels = labels.cuda()

            outputs = net(Variable(inputs))
            predicted = outputs.data

            # Important: Reduce dimensionality of tensor
            # Otherwise your MAE gets screwed up
            predicted = predicted.view(-1)

            # Compute mean absolute error
            diff = predicted - labels
            diff = diff.abs()
            diff = diff.sum()

            total_diff += diff
            total_nr += labels.size(0)

        mae = total_diff / total_nr

        print('Mean Absolute Error: %f' % mae)

        # Keep track of the performance to report on performance
        performance_statistic.append((running_loss,mae))

    print('Finished Training')
    
    return performance_statistic 

In [8]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import torchvision.datasets
import torchvision.models as models


transformsData = transforms.Compose([transforms.Scale(250), transforms.ToTensor()] )

dataset = UCSD( data_dir='/home/ec2-user/ml_data/uni/dbs/ucsd/ucsdpeds/vidf', 
                annotation_dir='/home/ec2-user/ml_data/uni/dbs/ucsd/vidf-cvpr', 
                transform=transformsData)

sampler = RandomSubsetSampler(dataset)

trainloader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=sampler.trainSampler(), num_workers=2 )
testloader = torch.utils.data.DataLoader(dataset, batch_size=1, sampler=sampler.testSampler(), num_workers=2)


/opt/anaconda3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [ ]:
# Evaluate performance of network 1
net1 = CountingCNN(
    regression_layer=regression_layer1
)
net1.cuda()
performance_statistic = evaluate_cnn_performance(net1, max_epoch=100)

# Free memory
del net1

Start training
0: 180223.32270818783
Mean Absolute Error: 4.823419
1: 55480.258019303845
Mean Absolute Error: 4.848201
2: 48593.40744354205
Mean Absolute Error: 4.498260
3: 44003.711197472905
Mean Absolute Error: 4.192979
4: 38851.954221126085
Mean Absolute Error: 3.778312
5: 35653.14156527475
Mean Absolute Error: 3.668431
6: 32831.601036374515
Mean Absolute Error: 3.361728
7: 30731.249469699735
Mean Absolute Error: 3.152181
8: 28491.964405523995
Mean Absolute Error: 3.130509
9: 26599.191267182963
Mean Absolute Error: 2.944230
10: 25317.01302279868
Mean Absolute Error: 2.942175
11: 24405.49544734317
Mean Absolute Error: 2.801876
12: 23075.698747384566
Mean Absolute Error: 2.671944
13: 21389.659197947247
Mean Absolute Error: 2.701888
14: 21160.980486515647
Mean Absolute Error: 2.585849
15: 20070.69431313617
Mean Absolute Error: 2.522069
16: 18757.7297682598
Mean Absolute Error: 2.428990
17: 18488.91395787306
Mean Absolute Error: 2.412328
18: 18233.343036128364
Mean Absolute Error: 2.414

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

loss_lst, mae_lst = zip(*performance_statistic)

loss_lst = list(loss_lst)
epoch_lst = list(range(len(loss_lst)))
mae_lst = list(mae_lst)

max_acc = 100
max_epoch = len(epoch_lst)
max_mae = max(mae_lst)
max_loss = 100000
plt.figure()
plt.plot(epoch_lst, loss_lst, 'g', linewidth=2.0)
plt.axis([0,max_epoch,0,max_loss])

plt.figure()
plt.plot(epoch_lst, mae_lst, 'g', linewidth=2.0)
plt.axis([0,max_epoch,0,max_mae])